In [11]:
import numpy as np
import pandas as pd
import sys
import os
import math as mth

In [2]:
pathToExcel = "E:\Github\Ladder-Iterative-Loadflow\Documentations\Datasets\slackbus.xlsx"
tolerance = 10e-4

In [3]:
def checkIfAllTablesExist(arr):
    if "BUS DATA FOLLOWS" or "BRANCH DATA FOLLOWS" in arr:
        return True
    else:
        return False

In [4]:
def extractData(npArr, txt):

    lenOfArray = npArr.shape[0]
    widthOfArray = npArr.shape[1]
    forLoopStartValue = int(np.where(npArr==txt)[0])+1
    for i in range(forLoopStartValue, lenOfArray):
        if npArr[i][0] == -999:
            newArray = npArr[forLoopStartValue:i]
            break

    if txt == "BUS DATA FOLLOWS":
        # Add code here for extracting SBase
        arrayToReturn = np.zeros((newArray.shape[0],5))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,7]
        arrayToReturn[:,2] = newArray[:,8]
        arrayToReturn[:,3] = newArray[:,11]
        arrayToReturn[:,4] = newArray[:,4]
    elif txt == "BRANCH DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,1]
        arrayToReturn[:,2] = newArray[:,6]
        arrayToReturn[:,3] = newArray[:,7]
        
    
    return arrayToReturn


In [5]:
def branchReorder(branchData, busData):
    newData = branchData
    lenOfArray = newData.shape[0]
    zerosCol = np.zeros((newData.shape[0],1))
    newData = np.c_[newData, zerosCol]
    newBranchDataArr = np.zeros((newData.shape[0],4))
    n=0
    duplicateRan = False
    
    i = 0
    while i < lenOfArray:
        
        dup = np.where(newData[:,0]==newData[i,0])
        if len(dup[0])==1 and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1            
        else:
            for j in dup[0]:
                k = j
                if (j > i) and (newData[k,4]==0):
                    
                    while(1):
                        if k+1 == lenOfArray:
                            if (newData[k,0]==newData[k-1,1]):
                                newBranchDataArr[n,0] = newData[k,0]
                                newBranchDataArr[n,1] = newData[k,1]
                                newBranchDataArr[n,2] = newData[k,2]
                                newBranchDataArr[n,3] = newData[k,3]
                                newData[k,4] = 1
                                n = n+1
                                k = k+1
                            break
                        if (newData[k,1]==newData[k+1,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                        elif (newData[k,1]!=newData[k+1,0]) and (newBranchDataArr[n-1,0] != newData[k,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                            break
                    duplicateRan = True
        if duplicateRan == True and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1
            duplicateRan = False
        
        i = i + 1
    return newBranchDataArr

In [84]:
#def dataParser(path):
path = pathToExcel
readInput = pd.read_excel(path, header=None)
numpyConversion = readInput.to_numpy()
checker = checkIfAllTablesExist(readInput)

if checker == False:
    raise ValueError("Excel does not contain Bus or Branch data, please double check and try again")

busData = extractData(numpyConversion, "BUS DATA FOLLOWS")
branchData = extractData(numpyConversion, "BRANCH DATA FOLLOWS")

sortedBranchData = branchReorder(branchData, busData)

# To display
x = pd.DataFrame(sortedBranchData)
display(x)




,0,1,2,3
0,1.0,2.0,0.0575,0.0293
1,2.0,19.0,0.1023,0.0976
2,19.0,20.0,0.9385,0.8457
3,20.0,21.0,0.2555,0.2985
4,21.0,22.0,0.4423,0.5848
5,2.0,3.0,0.3076,0.1567
6,3.0,23.0,0.2815,0.1924
7,23.0,24.0,0.5603,0.4424
8,24.0,25.0,0.5590,0.4374
9,3.0,4.0,0.2284,0.1163


In [82]:
inputArr = outputArr
busArr = busData
# Generate the load currents
for i in range(len(inputArr)-1,-1,-1):
    busFinder = np.where(busArr[:, 0] == inputArr[i, 1])
    j = busFinder[0][0]
    S = complex(busArr[j,1], busArr[j,2])
    inputArr[i, 5] = ((S)/inputArr[i, 4]).conjugate()
    #print(inputArr[i, 5])

# Calculate the currents for each bus
for i in range(len(inputArr)-1,-1,-1):
    dup = np.where(inputArr[:, 0] == inputArr[i, 1])
    #print(dup[0])
    if len(dup[0]) <= 1:
        if i == len(inputArr) - 1:
            inputArr[i, 6] = inputArr[i, 5] + 0 # Final bus, so all current goes into load
            #print(inputArr[i, 1])
        elif inputArr[i, 1] == inputArr[i + 1, 0]:
            inputArr[i, 6] = inputArr[i, 5] + inputArr[i + 1, 6]
            #print(inputArr[i, 1])
    elif len(dup[0]) > 1:
        k = dup[0][0] - 1
        if k != i:
            continue
        else:
            for j in dup[0]:
                if j != i:
                    inputArr[k, 6] = inputArr[k, 6] + inputArr[j, 6]
            inputArr[k, 6] = inputArr[i, 6] + inputArr[i, 5]
            
            
    #if i == len(inputArr) - 1:
        #inputArr[i, 6] = inputArr[i, 5] + 0 # Final bus, so all current goes into load
        #print(i)
                
    
    #else:
        
    #if i == len(inputArr)-1:
    #    inputArr[i, 6] = inputArr[i, 5] + 0
    #else:
    #    inputArr[i, 6] = inputArr[i, 5] + inputArr[i+1, 6]
        
#return(inputArr)

12
11
(26+0j)
20
11
(7+0j)
6
5
(23+0j)
9
5
(4+0j)
1
0
(19+0j)
5
0
(3+0j)


array([1, 5], dtype=int64)

In [50]:
def FWR(inputArr, Vs):
   for i in range(len(inputArr)):
      ZZ = complex(inputArr[i,2], inputArr[i,3])
      dup = np.where(inputArr[:, 0] == inputArr[i, 0])
      if len(dup[0]) == 1:
         if i == 0:
            inputArr[i, 4] = Vs - (ZZ*inputArr[i, 6]) # voltage at node 2
         # If the current row is part of the previous row's branch
         elif inputArr[i, 0] == inputArr[i-1, 1]:
            inputArr[i, 4] = inputArr[i - 1, 4] - (ZZ*inputArr[i, 6]) # voltage at node
            #print(inputArr[i, 0])
         #else:
      # If the current row has a duplicate somewhere
      elif len(dup[0]) > 1:
         # Iterate through the entire list of duplicates
         for j in dup[0]:
            #print(dup[0])
            # If the current duplicate is also the current bus
            if j == i:
                  # Check if we are still part of the same branch
                  if inputArr[j, 0] == inputArr[j - 1, 1]:
                     inputArr[j, 4] = inputArr[j - 1, 4] - (ZZ*inputArr[j, 6]) # voltage at node
                     #print(inputArr[i, 0])
                     #print(j)
                  else:
                     # Pull the bus number of the first duplicate, to get the voltage at that point
                     k = dup[0][0]
                     # Verify that the duplicate's previous row is still part of same branch
                     if inputArr[k, 0] == inputArr[k - 1, 1]:
                        inputArr[j, 4] = inputArr[k - 1, 4] - (ZZ*inputArr[j, 6]) # voltage at node
                        #print(inputArr[j, 0])
                        #print(k)
                        #print(j)
                        #print("")


   return inputArr

In [48]:
sortedBranchData
busArr = busData
Tol = 0.0001
Sb = 100*1000000
Vb = busData[0, 3] * 1000
Zb = (Vb**2)/Sb
print(Zb)
branchArr = sortedBranchData

outputArr = np.zeros([branchArr.shape[0], 7], dtype=np.complex_)
outputArr[:,0] = branchArr[:,0] # From Node
outputArr[:,1] = branchArr[:,1] # To Node
outputArr[:,2] = branchArr[:,2]*Zb # R
outputArr[:,3] = branchArr[:,3]*Zb # X
busArr[:, 1] = busArr[:, 1] * Sb
busArr[:, 2] = busArr[:, 2] * Sb
n = 0
Vold = 0
Vs = 1 * Vb

outputArr = FWR(outputArr, Vs)
Vl = outputArr[len(outputArr)-1, 4]
Err = ((abs(Vl.real - Vold.real))/Vs)

if Err.real<=Tol:
    print("Done")
else:
    Vold = Vl
    #outputArr = BKWsweep(busData, outputArr)


# To display
x = pd.DataFrame(outputArr)
display(x)

1.602756


,0,1,2,3,4,5,6
0,1.0+0.0j,2.0+0.0j,0.092158+0.000000j,0.046961+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
1,2.0+0.0j,19.0+0.0j,0.163962+0.000000j,0.156429+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
2,19.0+0.0j,20.0+0.0j,1.504187+0.000000j,1.355451+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
3,20.0+0.0j,21.0+0.0j,0.409504+0.000000j,0.478423+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
4,21.0+0.0j,22.0+0.0j,0.708899+0.000000j,0.937292+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
5,2.0+0.0j,3.0+0.0j,0.493008+0.000000j,0.251152+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
6,3.0+0.0j,23.0+0.0j,0.451176+0.000000j,0.308370+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
7,23.0+0.0j,24.0+0.0j,0.898024+0.000000j,0.709059+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
8,24.0+0.0j,25.0+0.0j,0.895941+0.000000j,0.701045+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j
9,3.0+0.0j,4.0+0.0j,0.366069+0.000000j,0.186401+0.000000j,12660.0+0.0j,0.0+0.0j,0.0+0.0j


In [9]:
busData
branchData
res = np.where(busData[:,4]==3)
if res[0][0] != 0:
    newBusStart = int(busData[res, 0][0][0])
       
    newData = branchData
    lenOfArray = newData.shape[0]
    zerosCol = np.zeros((newData.shape[0],1))
    newData = np.c_[newData, zerosCol]
    newBranchDataArr = np.zeros((newData.shape[0],4))
    n=0
    duplicateRan = False
    
    i = 0
    i = int(np.where(newData[:,0]==newBusStart)[0][0])
    while i < lenOfArray:
        dup = np.where(newData[:,0]==newData[i,0])
        if len(dup[0])==1 and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1
        else:
            for j in dup[0]:
                k = j
                if (j > i) and (newData[k,4]==0):
                    
                    while(1):
                        if k+1 == lenOfArray:
                            if (newData[k,0]==newData[k-1,1]):
                                newBranchDataArr[n,0] = newData[k,0]
                                newBranchDataArr[n,1] = newData[k,1]
                                newBranchDataArr[n,2] = newData[k,2]
                                newBranchDataArr[n,3] = newData[k,3]
                                newData[k,4] = 1
                                n = n+1
                                k = k+1
                            break
                        if (newData[k,1]==newData[k+1,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                        elif (newData[k,1]!=newData[k+1,0]) and (newBranchDataArr[n-1,0] != newData[k,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                            break
                    duplicateRan = True
        if duplicateRan == True and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1
            duplicateRan = False
        i = i + 1
    
    
    # To display
    x = pd.DataFrame(newBranchDataArr)
    display(x)


(array([0], dtype=int64),)
